In [2]:
import requests
from html import unescape
import re
from bs4 import BeautifulSoup

In [3]:
url = 'https://www.reuters.com/arc/outboundfeeds/news-sitemap-index/?outputType=xml'

In [4]:
res = requests.get(url)
if res.status_code != 200:
    print('ohho')

soup = BeautifulSoup(res.text, 'xml')

sitemap_tags = soup.find_all('sitemap')

sitemaps = []
for sitemap in sitemap_tags:
    loc = sitemap.find('loc')
    sitemaps.append(loc.text)

In [5]:
sitemaps

['https://www.reuters.com/arc/outboundfeeds/news-sitemap/?outputType=xml',
 'https://www.reuters.com/arc/outboundfeeds/news-sitemap/?outputType=xml&from=100',
 'https://www.reuters.com/arc/outboundfeeds/news-sitemap/?outputType=xml&from=200',
 'https://www.reuters.com/arc/outboundfeeds/news-sitemap/?outputType=xml&from=300',
 'https://www.reuters.com/arc/outboundfeeds/news-sitemap/?outputType=xml&from=400',
 'https://www.reuters.com/arc/outboundfeeds/news-sitemap/?outputType=xml&from=500',
 'https://www.reuters.com/arc/outboundfeeds/news-sitemap/?outputType=xml&from=600',
 'https://www.reuters.com/arc/outboundfeeds/news-sitemap/?outputType=xml&from=700',
 'https://www.reuters.com/arc/outboundfeeds/news-sitemap/?outputType=xml&from=800',
 'https://www.reuters.com/arc/outboundfeeds/news-sitemap/?outputType=xml&from=900',
 'https://www.reuters.com/arc/outboundfeeds/news-sitemap/?outputType=xml&from=1000']

In [6]:
urls = []
for sitemap in sitemaps:
    response = requests.get(sitemap)
    soup = BeautifulSoup(response.text, 'xml')
    url_tags = soup.find_all('url')
    
    for url in url_tags:
        loc = url.find('loc')
        urls.append(loc.text)

In [8]:
url = urls[-3]

In [9]:
response = requests.get(url)

In [10]:
soup = BeautifulSoup(response.text, 'html.parser')

In [12]:
url

'https://www.reuters.com/world/europe/russias-war-ukraine-latest-putin-gives-nuclear-warning-biden-rallies-allies-2023-02-21/'

In [11]:
for p in soup.select('p'):
    print(p.text)

[1/8] Ukrainian service members prepare to shoot from a howitzer at a front line, as Russia's attack on Ukraine continues, near the city of Bakhmut, Donetsk region, Ukraine March 2, 2023.  REUTERS/Oleksandr Ratushniak
March 3 (Reuters) - Russian troops and mercenaries were closing off the last access routes to the besieged Ukrainian city of Bakhmut on Friday, on the cusp of Moscow's first major victory in half a year after the bloodiest fighting of the war.
* Yevgeny Prigozhin, founder of Russia's Wagner mercenary force, said in a video that Bakhmut was "practically surrounded" by his forces and that Kyiv's forces had only one road left out.
* The commander of a Ukrainian drone unit active in Bakhmut, Robert Brovdi who goes by the name "Madyar", said in a video that his unit had been ordered by the military to withdraw immediately from the city. He gave no reason for the order.
* Russia said it would take measures to prevent new border incursions, a day after accusing Ukraine-backed na

In [13]:
def url_to_text(url):
    try:
        response = requests.get(url)
        if response.status_code != 200:
            return None
        
        soup = BeautifulSoup(response.text, 'html.parser')

        full_article = ''
        text = []
        heading = soup.select('h1')[0].text.strip()
        text.append(heading)
        
        for p in soup.select('p'):
            text.append(unescape(p.text))
        full_article = ' '.join(text)
        
        return (url, full_article) 
    except requests.exceptions.Timeout as ee :
        return None


In [14]:
import psycopg2

# Connect to the PostgreSQL database
conn = psycopg2.connect(
    host="localhost",
    database="SearchEngine",
    user="postgres",
    password="2580"
)

# Create a cursor object
cursor = conn.cursor()

# Open a cursor to perform database operations
cur = conn.cursor()


for url in urls:
    try:
        url, text = url_to_text(url)
        data = (url, text)
        sql = sql = "INSERT INTO sites (url, content) VALUES (%s, %s)"
        cur.execute(sql, data)
        conn.commit()
    except TypeError as ee :
        continue


KeyboardInterrupt: 